In [10]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report

# Load dataset
# Use the correct relative path to the Data folder
data = pd.read_csv("../Data/heart_2020_cleaned.csv")

# Encode all object (string) columns automatically
le = LabelEncoder()
for col in data.select_dtypes(include=['object']).columns:
    data[col] = le.fit_transform(data[col])

# Drop rows with missing values if any
data = data.dropna()

data.head()

,HeartDisease,BMI,Smoking,AlcoholDrinking,Stroke,PhysicalHealth,MentalHealth,DiffWalking,Sex,AgeCategory,Race,Diabetic,PhysicalActivity,GenHealth,SleepTime,Asthma,KidneyDisease,SkinCancer
0,0,16.60,1,0,0,3,30,0,0,7,5,2,1,4,5,1,0,1
1,0,20.34,0,0,1,0,0,0,0,12,5,0,1,4,7,0,0,0
2,0,26.58,1,0,0,20,30,0,1,9,5,2,1,1,8,1,0,0
3,0,24.21,0,0,0,0,0,0,0,11,5,0,0,2,6,0,0,1
4,0,23.71,0,0,0,28,0,1,0,4,5,0,1,4,8,0,0,0


In [11]:
# Features & Target
X = data.drop('HeartDisease', axis=1)
y = data['HeartDisease']  # Already encoded as 0/1

# Ensure all features are numeric (for safety)
if not all([pd.api.types.is_numeric_dtype(X[col]) for col in X.columns]):
    print("Non-numeric columns detected. Converting...")
    for col in X.columns:
        if not pd.api.types.is_numeric_dtype(X[col]):
            X[col] = pd.factorize(X[col])[0]

# Print dtypes for debugging
print("Feature dtypes:\n", X.dtypes)

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Evaluate
y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Feature dtypes:
 BMI                 float64
Smoking               int64
AlcoholDrinking       int64
Stroke                int64
PhysicalHealth        int64
MentalHealth          int64
DiffWalking           int64
Sex                   int64
AgeCategory           int64
Race                  int64
Diabetic              int64
PhysicalActivity      int64
GenHealth             int64
SleepTime             int64
Asthma                int64
KidneyDisease         int64
SkinCancer            int64
dtype: object
Accuracy: 0.9042355258837693
              precision    recall  f1-score   support

           0       0.92      0.98      0.95     58367
           1       0.36      0.12      0.18      5592

    accuracy                           0.90     63959
   macro avg       0.64      0.55      0.56     63959
weighted avg       0.87      0.90      0.88     63959



In [ ]:
# This cell assumes you have already run the data preparation and model training cells above.
# If you have not, please run those cells first.

import tkinter as tk
from tkinter import messagebox
import numpy as np
import sys
import threading
import time

# --- Professional Modern GUI Section ---
def launch_gui():
    missing_vars = []
    for var in ["X", "data", "model"]:
        if var not in globals():
            missing_vars.append(var)
    if missing_vars:
        print(f"Error: Please make sure you have run the data preparation and model training cells first. Missing: {', '.join(missing_vars)}")
        return
    feature_hints = {}
    feature_types = {}
    feature_ranges = {}
    for col in X.columns:
        unique_vals = sorted(data[col].unique())
        if len(unique_vals) == 2 and set(unique_vals) <= {0, 1}:
            feature_types[col] = 'bool'
            feature_hints[col] = 'Yes/No'
        elif np.issubdtype(data[col].dtype, np.integer) and len(unique_vals) <= 12:
            feature_types[col] = 'cat'
            feature_hints[col] = f"Options: {unique_vals}"
        elif np.issubdtype(data[col].dtype, np.integer) or np.issubdtype(data[col].dtype, np.floating):
            feature_types[col] = 'num'
            feature_ranges[col] = (float(np.nanmin(data[col])), float(np.nanmax(data[col])))
            feature_hints[col] = f"Range: {feature_ranges[col][0]} - {feature_ranges[col][1]}"
        else:
            feature_types[col] = 'entry'
            feature_hints[col] = 'Enter a value'

    # --- Color Palette ---
    BG_COLOR = "#f4f7fa"
    CARD_BG = "#ffffff"
    NAV_BG = "#1976d2"
    NAV_FG = "#ffffff"
    BUTTON_COLOR = "#1976d2"
    BUTTON_HOVER = "#1565c0"
    CLEAR_COLOR = "#e0e0e0"
    CLEAR_HOVER = "#bdbdbd"
    ENTRY_FOCUS = "#e3f2fd"
    ENTRY_BG = "#f9f9f9"
    BORDER_COLOR = "#bdbdbd"
    SHADOW = "#b0bec5"
    RESULT_BG = "#f5f5f5"
    RESULT_BORDER = "#1976d2"
    TOOLTIP_BG = "#333333"
    TOOLTIP_FG = "#ffffff"

    # --- Tooltip Widget ---
    class ToolTip(object):
        def __init__(self, widget, text):
            self.widget = widget
            self.text = text
            self.tipwindow = None
            widget.bind("<Enter>", self.show_tip)
            widget.bind("<Leave>", self.hide_tip)
        def show_tip(self, event=None):
            if self.tipwindow or not self.text:
                return
            x, y, cx, cy = self.widget.bbox("insert") if hasattr(self.widget, 'bbox') else (0,0,0,0)
            x = x + self.widget.winfo_rootx() + 40
            y = y + self.widget.winfo_rooty() + 20
            self.tipwindow = tw = tk.Toplevel(self.widget)
            tw.wm_overrideredirect(True)
            tw.wm_geometry(f"+{x}+{y}")
            label = tk.Label(tw, text=self.text, justify=tk.LEFT,
                             background=TOOLTIP_BG, foreground=TOOLTIP_FG,
                             relief=tk.SOLID, borderwidth=1,
                             font=("Segoe UI", 9, "normal"), padx=7, pady=3)
            label.pack(ipadx=1)
        def hide_tip(self, event=None):
            tw = self.tipwindow
            self.tipwindow = None
            if tw:
                tw.destroy()

    # --- Subtle Animated Result Popup ---
    def show_animated_result(text, color):
        popup = tk.Toplevel(root)
        popup.overrideredirect(True)
        popup.configure(bg=RESULT_BG, highlightthickness=2, highlightbackground=RESULT_BORDER)
        popup.geometry(f"340x70+{root.winfo_x()+100}+{root.winfo_y()+200}")
        popup.attributes("-topmost", True)
        popup.lift()
        popup.attributes("-alpha", 0.0)
        lbl = tk.Label(popup, text=text, font=("Segoe UI", 15, "bold"), fg=color, bg=RESULT_BG)
        lbl.pack(expand=True, fill="both", padx=10, pady=18)
        # Slide in and fade in
        for i in range(0, 11):
            popup.attributes("-alpha", i/10)
            popup.geometry(f"340x70+{root.winfo_x()+100}+{root.winfo_y()+200-i*8}")
            popup.update()
            time.sleep(0.02)
        # Hold
        time.sleep(1.2)
        # Fade out
        for i in range(10, -1, -1):
            popup.attributes("-alpha", i/10)
            popup.update()
            time.sleep(0.02)
        popup.destroy()

    # --- Entry Focus/Highlight ---
    def on_entry_focus_in(event):
        event.widget.config(bg=ENTRY_FOCUS, highlightbackground=BUTTON_COLOR, highlightcolor=BUTTON_COLOR, highlightthickness=2)
    def on_entry_focus_out(event):
        event.widget.config(bg=ENTRY_BG, highlightthickness=1, highlightbackground=BORDER_COLOR)

    # --- Predict Function ---
    def predict():
        try:
            vals = []
            for i, (ftype, widget) in enumerate(zip(types, widgets)):
                if ftype == 'bool':
                    val = widget.get()
                elif ftype == 'num':
                    val = widget.get()
                elif ftype == 'cat':
                    val = widget.get()
                else:
                    val = widget.get()
                if val == '' or val is None:
                    raise ValueError(f"Empty input for '{labels[i]}'")
                vals.append(float(val))
            vals = np.array(vals).reshape(1, -1)
            pred = model.predict(vals)[0]
            result = "High Risk of Heart Disease" if pred == 1 else "Low Risk"
            color = "#e53935" if pred == 1 else "#388e3c"
            threading.Thread(target=show_animated_result, args=(result, color), daemon=True).start()
        except Exception as ex:
            threading.Thread(target=show_animated_result, args=(f"Error: {ex}", "#e53935"), daemon=True).start()

    # --- Clear Function ---
    def clear_fields():
        for ftype, widget in zip(types, widgets):
            if ftype == 'bool':
                widget.set(0)
            elif ftype == 'num':
                widget.set(feature_ranges[labels[widgets.index(widget)]][0])
            elif ftype == 'cat':
                widget.set(options_map[labels[widgets.index(widget)]][0])
            else:
                widget.delete(0, tk.END)

    # --- About Dialog ---
    def show_about():
        messagebox.showinfo("About", "Heart Disease Predictor\n\nThis app predicts the risk of heart disease based on patient data using a machine learning model.\n\nDeveloped with Python, scikit-learn, and Tkinter.")

    # --- Main Window ---
    root = tk.Tk()
    root.title("Heart Disease Predictor")
    root.geometry("540x760")
    root.configure(bg=BG_COLOR)
    root.resizable(False, True)
    # Fade-in effect
    try:
        root.attributes("-alpha", 0.0)
        for i in range(0, 11):
            root.attributes("-alpha", i/10)
            root.update()
            time.sleep(0.02)
    except Exception:
        pass

    # --- Navigation Bar ---
    nav = tk.Frame(root, bg=NAV_BG, height=54)
    nav.pack(fill="x", side="top")
    nav.grid_propagate(False)
    nav.columnconfigure(0, weight=1)
    nav.columnconfigure(1, weight=0)
    title_lbl = tk.Label(nav, text="Heart Disease Predictor", font=("Segoe UI", 18, "bold"), fg=NAV_FG, bg=NAV_BG)
    title_lbl.grid(row=0, column=0, sticky="w", padx=22, pady=10)
    about_btn = tk.Button(nav, text="About", command=show_about, font=("Segoe UI", 10, "bold"), bg=NAV_BG, fg=NAV_FG, activebackground="#145ea8", activeforeground=NAV_FG, relief="flat", bd=0, padx=12, pady=2, cursor="hand2")
    about_btn.grid(row=0, column=1, sticky="e", padx=18, pady=10)

    # --- Card Frame (with shadow) ---
    shadow = tk.Frame(root, bg=SHADOW, bd=0, highlightthickness=0)
    shadow.place(relx=0.5, rely=0.09+0.012, anchor="n", width=500, height=670)
    card = tk.Frame(root, bg=CARD_BG, bd=0, highlightthickness=1, highlightbackground=BORDER_COLOR)
    card.place(relx=0.5, rely=0.09, anchor="n", width=500, height=670)
    card.lift()
    threading.Thread(target=animate_card_border, args=(card,), daemon=True).start()

    subtitle = tk.Label(card, text="Enter patient data below:", font=("Segoe UI", 12), bg=CARD_BG, fg="#616161")
    subtitle.pack(pady=(18, 10))

    # --- Scrollable Input Area ---
    canvas = tk.Canvas(card, bg=CARD_BG, highlightthickness=0, width=480, height=440)
    scrollbar = tk.Scrollbar(card, orient="vertical", command=canvas.yview)
    scroll_frame = tk.Frame(canvas, bg=CARD_BG)
    scroll_frame.bind(
        "<Configure>", lambda e: canvas.configure(scrollregion=canvas.bbox("all"))
    )
    canvas.create_window((0, 0), window=scroll_frame, anchor="nw")
    canvas.configure(yscrollcommand=scrollbar.set)
    canvas.pack(side="left", fill="both", expand=True, padx=(10,0), pady=(0,0))
    scrollbar.pack(side="right", fill="y")

    labels = list(X.columns)
    widgets = []
    types = []
    options_map = {}
    for i, label in enumerate(labels):
        row = tk.Frame(scroll_frame, bg=CARD_BG)
        row.pack(fill="x", pady=4, padx=10)
        lbl = tk.Label(row, text=label, font=("Segoe UI", 11, "bold"), width=18, anchor="w", bg=CARD_BG, fg="#1976d2")
        lbl.pack(side="left")
        ftype = feature_types[label]
        types.append(ftype)
        # Special handling for Sex
        if label.lower() == 'sex':
            var = tk.StringVar(value='Male')
            options = ['Male', 'Female']
            dropdown = tk.OptionMenu(row, var, *options)
            dropdown.config(font=("Segoe UI", 11), bg=ENTRY_BG, fg="#1976d2", highlightthickness=0, width=10)
            dropdown.pack(side="left", padx=(0, 8))
            widgets.append(var)
            ToolTip(dropdown, "Select Male or Female")
        # Special handling for AgeCategory
        elif label.lower() == 'agecategory' or label.lower() == 'age':
            # Age bins: 10-20, 21-30, ..., 91-100
            age_bins = [f"{i}-{i+10}" for i in range(10, 100, 10)]
            var = tk.StringVar(value=age_bins[0])
            dropdown = tk.OptionMenu(row, var, *age_bins)
            dropdown.config(font=("Segoe UI", 11), bg=ENTRY_BG, fg="#1976d2", highlightthickness=0, width=10)
            dropdown.pack(side="left", padx=(0, 8))
            widgets.append(var)
            ToolTip(dropdown, "Select age range (10-100)")
        elif ftype == 'bool':
            var = tk.IntVar(value=0)
            yes_btn = tk.Radiobutton(row, text="Yes", variable=var, value=1, font=("Segoe UI", 11), bg=CARD_BG, fg="#388e3c", selectcolor=ENTRY_BG, activebackground=CARD_BG, activeforeground="#388e3c", highlightthickness=0, width=6, indicatoron=0, relief="ridge", bd=1)
            no_btn = tk.Radiobutton(row, text="No", variable=var, value=0, font=("Segoe UI", 11), bg=CARD_BG, fg="#e53935", selectcolor=ENTRY_BG, activebackground=CARD_BG, activeforeground="#e53935", highlightthickness=0, width=6, indicatoron=0, relief="ridge", bd=1)
            yes_btn.pack(side="left", padx=(0, 2))
            no_btn.pack(side="left", padx=(0, 8))
            widgets.append(var)
            ToolTip(yes_btn, feature_hints[label])
            ToolTip(no_btn, feature_hints[label])
        elif ftype == 'num' and label in feature_ranges and feature_ranges[label][1] - feature_ranges[label][0] <= 200:
            minv, maxv = feature_ranges[label]
            var = tk.DoubleVar(value=minv)
            scale = tk.Scale(row, variable=var, from_=minv, to=maxv, orient=tk.HORIZONTAL, resolution=1 if maxv-minv<20 else 0.1, length=180, bg=CARD_BG, fg="#1976d2", highlightthickness=0, sliderrelief="flat", troughcolor=ENTRY_BG, bd=1)
            scale.pack(side="left", padx=(0, 8))
            widgets.append(var)
            ToolTip(scale, feature_hints[label])
        elif ftype == 'cat':
            options = sorted(data[label].unique())
            var = tk.StringVar(value=str(options[0]))
            options_map[label] = [str(opt) for opt in options]
            dropdown = tk.OptionMenu(row, var, *options_map[label])
            dropdown.config(font=("Segoe UI", 11), bg=ENTRY_BG, fg="#1976d2", highlightthickness=0, width=10)
            dropdown.pack(side="left", padx=(0, 8))
            widgets.append(var)
            ToolTip(dropdown, feature_hints[label])
        else:
            entry = tk.Entry(row, font=("Segoe UI", 11), width=12, relief="flat", bd=2, bg=ENTRY_BG, highlightthickness=1, highlightbackground=BORDER_COLOR)
            entry.pack(side="left", padx=(0, 8))
            entry.bind("<FocusIn>", on_entry_focus_in)
            entry.bind("<FocusOut>", on_entry_focus_out)
            widgets.append(entry)
            ToolTip(entry, feature_hints[label])
        # Add focus/hover animation to row
        def make_row_focus_anim(row=row):
            def on_focus_in(event):
                animate_row_highlight(row, True)
            def on_focus_out(event):
                animate_row_highlight(row, False)
            return on_focus_in, on_focus_out
        for child in row.winfo_children():
            if isinstance(child, tk.Entry) or isinstance(child, tk.OptionMenu) or isinstance(child, tk.Scale):
                on_in, on_out = make_row_focus_anim(row)
                child.bind('<FocusIn>', on_in)
                child.bind('<FocusOut>', on_out)
        # For OptionMenu, animate on click
        for child in row.winfo_children():
            if isinstance(child, tk.OptionMenu):
                def on_menu(event, child=child):
                    child.config(bg="#e3f2fd")
                    child.after(200, lambda: child.config(bg=ENTRY_BG))
                child.bind('<Button-1>', on_menu)

    # --- Button Row ---
    btn_row = tk.Frame(card, bg=CARD_BG)
    btn_row.pack(pady=28)
    predict_btn = tk.Button(btn_row, text="Predict", command=lambda: [animate_button_press(predict_btn), predict()], font=("Segoe UI", 13, "bold"), bg=BUTTON_COLOR, fg="white", activebackground=BUTTON_HOVER, activeforeground="white", relief="groove", bd=2, width=14, height=2, cursor="hand2", highlightthickness=1, highlightbackground=BORDER_COLOR)
    predict_btn.pack(side="left", padx=(0, 18))
    clear_btn = tk.Button(btn_row, text="Clear", command=lambda: [animate_button_press(clear_btn), clear_fields()], font=("Segoe UI", 13, "bold"), bg=CLEAR_COLOR, fg="#333", activebackground=CLEAR_HOVER, activeforeground="#333", relief="groove", bd=2, width=10, height=2, cursor="hand2", highlightthickness=1, highlightbackground=BORDER_COLOR)
    clear_btn.pack(side="left")

    # --- Button Hover Effects ---
    def on_btn_enter(e):
        e.widget.config(bg=BUTTON_HOVER if e.widget == predict_btn else CLEAR_HOVER)
    def on_btn_leave(e):
        e.widget.config(bg=BUTTON_COLOR if e.widget == predict_btn else CLEAR_COLOR)
    predict_btn.bind("<Enter>", on_btn_enter)
    predict_btn.bind("<Leave>", on_btn_leave)
    clear_btn.bind("<Enter>", on_btn_enter)
    clear_btn.bind("<Leave>", on_btn_leave)

    # --- Window Close: Stop Animations ---
    def on_close():
        root.destroy()
    root.protocol("WM_DELETE_WINDOW", on_close)

    root.mainloop()

if __name__ == "__main__" or "ipykernel" in sys.modules:
    print("Warning: Running a Tkinter GUI in a Jupyter notebook may block the kernel.\nTo use the GUI, call launch_gui() in a script or terminal, or run launch_gui() in a notebook cell when ready.")
    # Uncomment the next line to launch the GUI directly (not recommended in Jupyter):
    # launch_gui()


To use the GUI, call launch_gui() in a script or terminal, or run launch_gui() in a notebook cell when ready.


In [14]:
# To launch the GUI, simply run this cell after running the previous cells.
# This will work in Jupyter by running Tkinter in a separate thread and keeping the cell alive as long as the GUI is open.
import threading
import time
import sys

def run_gui_thread():
    try:
        launch_gui()
    except Exception as e:
        print(f"Error launching GUI: {e}")

# Try to create a simple Tkinter window to check GUI support
try:
    import tkinter as tk
    test_root = tk.Tk()
    test_root.withdraw()  # Hide the window
    test_root.update_idletasks()
    test_root.destroy()
    gui_supported = True
except Exception as e:
    gui_supported = False
    print(f"GUI not supported in this environment: {e}\nIf you are running on a remote server or headless environment, GUI windows may not be supported.")

if gui_supported:
    thread = threading.Thread(target=run_gui_thread)
    thread.start()
    # Keep the cell alive as long as the GUI is open
    try:
        while thread.is_alive():
            time.sleep(0.2)
    except KeyboardInterrupt:
        print("Interrupted by user. Closing GUI.")
    print("GUI closed. Cell finished.")
else:
    print("This environment does not support GUI windows. Please run this notebook on your local machine.")

GUI closed. Cell finished.
